In [196]:
%load_ext autoreload
%autoreload 2
from models import *
from names import *
import seaborn as sns
import matplotlib.pyplot as plt
import cairosvg


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:
results_df = aggregate_results_from_subfolders(".")

results_df.sort_values(by="difference of ACE", ascending=True, inplace=True)
# results_df.sort_values(by="pre_r_squared test", ascending=False, inplace=True)

results_df.to_csv("results.csv", index=False)
print(f"{len(results_df)} records")

63 records


In [198]:
# results_digest = results_df[
#     [
#         "precovid ACE",
#         "postcovid ACE",
#         "difference of ACE",
#         "outcome",
#         "treatment",
#     ]
# ].copy()
results_digest = results_df[
    [
        "difference of ACE",
        "outcome",
        "treatment",
    ]
].copy()
results_digest["outcome description"] = results_digest["outcome"].apply(
    lambda x: full_dictionary[x]
)
results_digest["treatment description"] = results_df["treatment"].apply(
    lambda x: full_dictionary[x]
)
result_digest = results_digest[
    ["treatment description", "outcome description", "difference of ACE"]
]

In [199]:
len(results_digest["outcome"].unique())

7

In [200]:
len(results_digest["treatment"].unique())

14

In [201]:
df_wide = results_digest.pivot(
    index="treatment description",
    columns="outcome description",
    values="difference of ACE",
)
df_wide

outcome description,gad2_score,pam,phq2_score,phq4_score,social_level,sse_score,stress
treatment description,,,,,,,
cohort year,NaN,0.210,NaN,NaN,-0.07660,NaN,NaN
distance traveled (meters),NaN,NaN,NaN,NaN,-0.13500,NaN,NaN
duration unlocked phone in social locations (minutes),0.2690,-0.570,0.0815,0.38000,NaN,-0.8750,0.18900
excercise (seconds),-0.0164,NaN,NaN,0.00335,0.16000,0.2230,-0.00736
frequency of unlocked phone in social locations,-0.1610,NaN,0.0687,0.09070,-0.09350,0.2950,-0.07460
gender,0.1790,NaN,0.3370,0.44400,NaN,NaN,NaN
in house (hours),-0.0668,0.628,-0.1590,-0.29500,NaN,-0.0481,-0.07430
motion at social locations (minutes),-0.1740,NaN,NaN,NaN,0.00667,NaN,NaN
race,0.1580,0.369,-0.0403,NaN,NaN,0.1840,NaN


In [202]:
import pdfkit


def color_gradient(val):
    # Normalize the value to the range [0, 1]
    norm = (val - df_wide.min().min()) / (
        df_wide.max().max() - df_wide.min().min()
    )
    # Use a colormap to get the color
    cmap = plt.get_cmap("coolwarm")
    # Get the color as a tuple (r, g, b, a)
    color = cmap(norm)
    return (
        f"background-color: rgba({int(color[0]*255)}, {int(color[1]*255)},"
        f"{int(color[2]*255)}, {color[3]}); color: black; "
        "font-family: Arial; font-size: 12px;font-weight: bold;"
    )


styled_df = df_wide.style.map(color_gradient)
# Replace NaN with an empty string
styled_df = styled_df.format(na_rep="")

# Add borders to the cells
styled_df = styled_df.set_table_styles(
    [{"selector": "th, td", "props": [("border", "1px solid black")]}]
)
# Convert the styled DataFrame to HTML
html = styled_df.to_html()

# Save the HTML to a file (optional, for debugging)
with open("styled_dataframe.html", "w") as file:
    file.write(html)

options = {
    "disable-smart-shrinking": "",
}

pdfkit.from_string(html, "styled_dataframe.pdf", options=options)

True